In [1]:
!mkdir /kaggle/working/train
!ln -s /kaggle/input/sp-cup-train-dataset/train_fake/fake /kaggle/working/train/fake
!ln -s /kaggle/input/sp-cup-train-dataset/train_real/real /kaggle/working/train/real

!mkdir /kaggle/working/test
!ln -s /kaggle/input/sp-cup-fake-validation/valid_fake/fake /kaggle/working/test/fake
!ln -s /kaggle/input/sp-cup-fake-validation/valid_real/real /kaggle/working/test/real

In [10]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.applications import EfficientNetV2B3
from kaggle_datasets import KaggleDatasets

# Step 1: Initialize TPU
# try:
#     tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # Detect TPU
#     tf.config.experimental_connect_to_cluster(tpu)
#     tf.tpu.experimental.initialize_tpu_system(tpu)
#     strategy = tf.distribute.TPUStrategy(tpu)
#     print("All TPU devices: ", tf.config.list_logical_devices('TPU'))
# except ValueError:
#     print("TPU not found.")

strategy = tf.distribute.OneDeviceStrategy(device="/gpu:0")
# mirrored_strategy = tf.distribute.MirroredStrategy(devices=["/gpu:0", "/gpu:1"])

# Use GPU

In [3]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


# Dataset 

In [4]:
import keras

In [6]:
image_size = (224, 224)
batch_size = 128

train_ds, val_ds = keras.utils.image_dataset_from_directory(
    "/kaggle/working/train",
    validation_split=0.2,
    subset="both",
    seed=1337,
    image_size=image_size,
    batch_size=batch_size,
)

Found 262160 files belonging to 2 classes.
Using 209728 files for training.
Using 52432 files for validation.


In [13]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Callbacks
callbacks = [
    EarlyStopping(
        monitor='val_loss',  # Monitor validation loss
        patience=40,          # Stop if no improvement for 5 epochs
        restore_best_weights=True  # Restore best weights after stopping
    ),
    ModelCheckpoint(
        "best_model.keras",  # Use `.keras` extension for saving the best model
        monitor='val_loss',  # Monitor validation loss
        save_best_only=True,  # Save only the best model
        save_weights_only=False,  # Save the entire model (not just weights)
        mode='auto'  # Automatically choose 'min' or 'max' based on the monitored metric
    )
]


In [16]:
with strategy.scope():
    model = tf.keras.Sequential([
        EfficientNetV2B3(include_top=False, input_shape=(224, 224, 3), pooling='avg'),
        layers.Dense(1, activation='sigmoid')
    ])

    model.compile(
        optimizer='adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )

# Step 4: Train Model
epochs = 100
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs,
    steps_per_epoch=len(train_ds) // batch_size,
    validation_steps=len(val_ds) // batch_size,
    callbacks=callbacks  # Add callbacks here

)

# Step 5: Save Model
model.save("efficientnet_cifake.h5")

52606240/52606240 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/100


2025-01-20 17:06:20.142610: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:961] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inStatefulPartitionedCall/sequential_1_1/efficientnetv2-b3_1/block1b_drop_1/stateless_dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


12/12 ━━━━━━━━━━━━━━━━━━━━ 79s 1s/step - accuracy: 0.6982 - loss: 0.5552 - val_accuracy: 0.8099 - val_loss: 0.4394
Epoch 2/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 11s 915ms/step - accuracy: 0.8266 - loss: 0.4204 - val_accuracy: 0.8255 - val_loss: 0.4691
Epoch 3/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 0.8483 - loss: 0.3493 - val_accuracy: 0.8438 - val_loss: 0.3943
Epoch 4/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 11s 915ms/step - accuracy: 0.8747 - loss: 0.3153 - val_accuracy: 0.8568 - val_loss: 0.4073
Epoch 5/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 0.8617 - loss: 0.3300 - val_accuracy: 0.8516 - val_loss: 0.3688
Epoch 6/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 0.8677 - loss: 0.2988 - val_accuracy: 0.8411 - val_loss: 0.3576
Epoch 7/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 11s 915ms/step - accuracy: 0.8604 - loss: 0.3107 - val_accuracy: 0.8620 - val_loss: 0.5635
Epoch 8/100
12/12 ━━━━━━━━━━━━━━━━━━━━ 13s 1s/step - accuracy: 0.8911 - loss: 0.2430 - val_accuracy: 0.8750 - val_

# On Test Dataset

In [17]:
import tensorflow as tf

# Define the same preprocessing steps
def preprocess_image(image, label):
    image = tf.image.resize(image, (224, 224))  # Resize for EfficientNet
    image = image / 255.0  # Normalize to [0, 1]
    return image, label

# Load test dataset
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    '/kaggle/working/test',
    image_size=(224, 224),  # Resize for EfficientNet
    batch_size=32,
    shuffle=False  # Do not shuffle for consistent evaluation
)

# Apply preprocessing
test_dataset = test_dataset.map(preprocess_image)


Found 3072 files belonging to 2 classes.


In [18]:
# Evaluate the model on the test dataset
test_loss, test_accuracy = model.evaluate(test_dataset)

96/96 ━━━━━━━━━━━━━━━━━━━━ 7s 65ms/step - accuracy: 0.8377 - loss: 0.5414
